In [2]:
from sympy import *
import itertools as it
import numpy as np

In [3]:
# Make symbols: methods to dynamically create as many as needed strings and Sympy symbols
# Shift is starting index; default 0

def make_strings(N,s, shift = 0):
	my_strings = []
	for i in range(shift, N + shift):
	    tmp_st = f'{s}{i}'
	    my_strings.append(tmp_st)
	return my_strings

def make_symbols(N,s, shift = 0):
	my_symbols = []
	for i in range(shift, N + shift):
	    tmp_st = f'{s}{i}'
	    globals()[tmp_st] = Symbol(tmp_st)
	    my_symbols.append(globals()[tmp_st])
	return my_symbols

# Generate harmonic game

Given measure, find harmonic game

# Parameters

In [4]:
# Game skeleton

#numPlayers = 4

#skeleton = np.random.randint(2, 6, numPlayers)

skeleton = [2, 2]
skeleton

[2, 2]

In [5]:
# N
numPlayers = len(skeleton)
players = range(numPlayers)

# -------------------------
# Number of action profiles
# -------------------------

# A
numPures = prod(skeleton)

# List of A_{-i} = for each player, number of action profiles of other players
numPuresMinus = [  int(numPures / skeleton[i]) for i in players ]

# -------------------------
# Number of payoff degrees of freedom
# -------------------------

# AN; number of payoff degrees of freedom
numPays = numPlayers * numPures

# sum_i A_i; number of harmonic measure dofs
numMeas = sum(skeleton)

In [6]:
# Pure actions: list of N lists, each with Ai elements; pure actions of each player

pures_play = [ make_strings(skeleton[i], f'a{i}', shift = 1) for i in players ]

In [7]:
pures_play

[['a01', 'a02'], ['a11', 'a12']]

In [8]:
# Pure profiles; cartesian product of pure actions of each player
# Returns one list with A = numPures elements; each element is a tuple of strings

pures = list(it.product(*pures_play))

In [9]:
pures

[('a01', 'a11'), ('a01', 'a12'), ('a02', 'a11'), ('a02', 'a12')]

In [10]:
assert len(pures) == numPures

# Harmonic measure

In [11]:
# mu_values = [ np.random.randint(1, 5, skeleton[i]) for i in players]

mu_values = [  [1, 2], [1, 1] ]

# mu_values = [  [1, 2], [1, 3], [1,4], [1, 2, 3]  ]

mu_values

[[1, 2], [1, 1]]

In [12]:
# Pack mu in list of dicts

mu = [   dict(zip( pures_play[i], mu_values[i] )) for i in players    ]
mu

[{'a01': 1, 'a02': 2}, {'a11': 1, 'a12': 1}]

In [13]:
numPures

4

In [14]:
numPuresMinus

[2, 2]

In [15]:
numPays

8

In [16]:
numMeas

4

In [17]:
def print_payoff(payoff_dict, payoff_symbol):
    for i in players:
        for a in pures:
            print( f'{payoff_symbol}_{i}{a} = {payoff_dict[i][a]}' )
        print()

# Profiles of other players

In [18]:
# Pure profiles of other players
# Make list of N lists; the list pure_minus[i] contains the pure action profiles of players other than i
# Build taking the cartesian product of pure actions of all players other than i
# The size of the list pure_minus[i] is A_{-i} = \prod_{j \neq i} A_j

pures_minus = [ list( it.product( *( pures_play[:i] + pures_play[i+1:] ) ) ) for i in players ]

In [19]:
pures_minus

[[('a11',), ('a12',)], [('a01',), ('a02',)]]

In [20]:
numPuresMinus

[2, 2]

In [21]:
for i in players:
    assert len(pures_minus[i]) == numPuresMinus[i]

# Make a

In [22]:
# Util to make (a_i, a_{-i}) given a_i and a_{-i} as tuple of strings (to be used as key for payoff dictionaries)
def make_pure(ai, a_minus_i, i):
    l = list(a_minus_i)
    return tuple(l[:i] + [ai] + l[i:])

In [23]:
def split_pure(a, i):
    
    ai = a[i]

    l = list(a)
    a_minus_i = tuple( l[:i] + l[i+1:] )
    return ai, a_minus_i

In [24]:
# check consistency
for i in players:
    for a in pures:
        ai, a_minus_i = split_pure(a, i)
        assert a == make_pure( ai, a_minus_i, i )

# Harmonic payoff

In [25]:
# harmonic payoff degrees of freedom; will build harmonic payoff h out of these

# to be determined harmonic payoff of player i (as many as A)

h_sym = [make_symbols(numPures, f'h{i}', shift = 1) for i in players]
h_sym

[[h01, h02, h03, h04], [h11, h12, h13, h14]]

In [26]:
h = [  dict(zip(pures , h_sym[i] ))  for i in players]
h

[{('a01', 'a11'): h01,
  ('a01', 'a12'): h02,
  ('a02', 'a11'): h03,
  ('a02', 'a12'): h04},
 {('a01', 'a11'): h11,
  ('a01', 'a12'): h12,
  ('a02', 'a11'): h13,
  ('a02', 'a12'): h14}]

In [27]:
pures

[('a01', 'a11'), ('a01', 'a12'), ('a02', 'a11'), ('a02', 'a12')]

# System

In [28]:
pures

[('a01', 'a11'), ('a01', 'a12'), ('a02', 'a11'), ('a02', 'a12')]

In [29]:
pures_play

[['a01', 'a02'], ['a11', 'a12']]

In [30]:
pures_minus

[[('a11',), ('a12',)], [('a01',), ('a02',)]]

In [31]:
# --------------------------------
# KEY: HARMONIC FUNCTION
# --------------------------------
def Hi(a, i):
    ai, a_minus_i = split_pure(a, i)
    return sum(   [   mu[i][bi] * ( h[i][a] - h[i][ make_pure(bi, a_minus_i, i) ] )    for bi in pures_play[i]   ]  )

In [32]:
for i in players:
    for a in pures:
        print( Hi( a, i ) )

2*h01 - 2*h03
2*h02 - 2*h04
-h01 + h03
-h02 + h04
h11 - h12
-h11 + h12
h13 - h14
-h13 + h14


In [33]:
# --------------------------------
# KEY: HARMONIC SYSTEM in degrees of freedom h, that is payoff to be determined, prescribed the harmonic measure
# --------------------------------
system = [  sum( [  Hi(a,i)   for i in players ] )   for a in pures  ]

In [34]:
system

[2*h01 - 2*h03 + h11 - h12,
 2*h02 - 2*h04 - h11 + h12,
 -h01 + h03 + h13 - h14,
 -h02 + h04 - h13 + h14]

In [35]:
# one equation per pure in pures
assert len(system) == numPures

In [36]:
# Unknowns of system to solve: the weights (one per player) + the non-strategic dofs
dofs = list(it.chain(*h_sym))
dofs

[h01, h02, h03, h04, h11, h12, h13, h14]

In [37]:
numDofs = len(dofs)

In [38]:
# Unknowns of system to solve: harmonic weights
assert numDofs == numPays

In [39]:
# Matrix of linear system
A, b = linear_eq_to_matrix(system, *dofs)

In [40]:
A

Matrix([
[ 2,  0, -2,  0,  1, -1,  0,  0],
[ 0,  2,  0, -2, -1,  1,  0,  0],
[-1,  0,  1,  0,  0,  0,  1, -1],
[ 0, -1,  0,  1,  0,  0, -1,  1]])

In [41]:
sol = solve(system, dofs)

In [42]:
# Solution
sol

{h01: h03 + h13 - h14, h02: h04 - h13 + h14, h11: h12 - 2*h13 + 2*h14}

In [43]:
# Replace sol in dofs

extracted_sol = [e.subs(sol) for e in dofs]

In [44]:
extracted_sol

[h03 + h13 - h14,
 h04 - h13 + h14,
 h03,
 h04,
 h12 - 2*h13 + 2*h14,
 h12,
 h13,
 h14]

# General solution here

In [45]:
# zip solution in dictionary with dofs
sol_dict = dict(zip(dofs, extracted_sol))

In [46]:
sol_dict

{h01: h03 + h13 - h14,
 h02: h04 - h13 + h14,
 h03: h03,
 h04: h04,
 h11: h12 - 2*h13 + 2*h14,
 h12: h12,
 h13: h13,
 h14: h14}

---

# Build solution instance

In [47]:
# fix remaining dofs; generate dictionary with dofs keys and random values
fix_remaining_dofs = dict(zip(dofs, np.random.randint(-5, 5, numDofs)))

In [48]:
fix_remaining_dofs

{h01: 0, h02: -3, h03: -2, h04: -1, h11: 1, h12: 0, h13: -1, h14: -3}

In [49]:
# fix remaining dofs to random number by subbing in sol_dict
sol_dict_instance = { key : sol_dict[key].subs(fix_remaining_dofs) for key in sol_dict}

In [50]:
sol_dict_instance

{h01: 0, h02: -3, h03: -2, h04: -1, h11: -4, h12: 0, h13: -1, h14: -3}

# Result: harmonic game

In [51]:
print(skeleton)

[2, 2]


In [52]:
sol_dict_instance.values()

dict_values([0, -3, -2, -1, -4, 0, -1, -3])

# End